# 概要
このノートブックは[Pytorch Bert baseline NBME by Shudipto Trafder](https://www.kaggle.com/iamsdt/pytorch-bert-baseline-nbme/notebook)を非常に参考にしながら，各コードの動作や意図についての私の理解やメモをまとめたものです．

私が上のノートブックを理解するまでに学んだことや調べたことを書いています．私と同じような初心者の方の参考になれば幸いです．

間違いの指摘や質問などあればコメントからお願いします．

もしこのノートブックを気に入っていただけたら，UP VOTEをお願いします🎉

In [ ]:
from ast import literal_eval
from itertools import chain
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer

# データの確認

In [ ]:
features = pd.read_csv("../input/nbme-score-clinical-patient-notes/features.csv")
notes = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")
train = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
test = pd.read_csv("../input/nbme-score-clinical-patient-notes/test.csv")

`patient_notes.csv`にはpatient notes（カルテ）が入っています．各カルテには`pn_num`（カルテ番号），`case_num`（症状番号）が割り振られています．

In [ ]:
notes

`features.csv`には，各caseごとに重要なfeature（特徴，症状）が入っています．`feature_text`はそのfeatureの説明で，例えば"Chest-pressure"（胸の圧迫感）や"Photophobia"（強い光を受けた際に不快感や眼の痛みなどを生じること）などがあります．

`feature_num`の先頭は`case_num`に対応しています（`case_num`が0の場合を除く）．

In [ ]:
features

`train.csv`には，カルテの中で各featureに対応する記述`annotation`とその位置`location`が含まれています．

またannotaionやlocationの中身は`"['dad with recent heart attcak']"`や`"['696 724']"`のような文字列になっています．


In [ ]:
train

例えば，`id`が`00016_002`の`annotaion`は`"chest pressure"`，`location`は`[203 217]`となっています．

実際，カルテの中から`location`の範囲を取り出してみると，`annnotaion`に一致しています．

In [ ]:
notes[notes["pn_num"]==16]["pn_history"].iloc[0][203:217]

featureによってはカルテ中にannotationが存在しない場合もあります．

逆に１つのfeatureに対して複数のannotaionがあったり，1つのannotationが複数箇所に別れて存在している場合もあります．

In [ ]:
train[(train["id"]=="00211_000")|(train["id"]=="00100_010")]

また，すべてのカルテに対してannotationが用意されているわけではないようです．`train.csv`に含まれている，つまりannotationが存在しているカルテの数を調べてみます．

In [ ]:
len(train["pn_num"].unique())

`test.csv`には予測対象のデータが入っています．各行にはカルテ番号`pn_num`，case番号`case_num`，feature番号`feature_num`が含まれています．

なおこのファイルはダミーで，提出時に本物のファイルに置き換えられます．

In [ ]:
test

ここではデータの中身やフォーマットを確認しました．詳細やEDAに関してはすでに様々なノートブックが存在しています．

* https://www.kaggle.com/drcapa/nbme-starter
* https://www.kaggle.com/utcarshagrawal/nbme-complete-eda
* https://www.kaggle.com/yufuin/nbme-japanese

この先，自然言語処理とHuggingFaceの各種ライブラリ（主にTransformers）の知識がある程度必要になってきます．

私は完全に素人ですが，このコンペに参加する直前に偶然読んでいた[HuggingFaceのコース](https://huggingface.co/course)のおかげで自然言語処理や🤗 Transformersの最低限の知識が得られました（得られた気がしています）．長すぎずわかりやすい内容なので非常におすすめです．

より詳細な説明は[🤗のドキュメント](https://huggingface.co/docs)や，[Natural Language Processing with Transformers](https://www.oreilly.com/library/view/natural-language-processing/9781098103231/)などが参考になります．

# 前処理・Dataset作成
## QA/NERハイブリッドアプローチ

前処理やDataset作成の前に，モデルの入出力を把握する必要があります．

このノートブックのベースになっている[Pytorch Bert baseline NBME](https://www.kaggle.com/iamsdt/pytorch-bert-baseline-nbme/notebook)では[QA/NER hybrid train 🚆](https://www.kaggle.com/nbroad/qa-ner-hybrid-train-nbme)のノートブックで紹介されているQA(Question Answering)とNER(Named Entity Recognition)を組み合わせたアプローチを利用しています．

このアプローチでは，feature_text（featureの説明）とカルテを分類モデルに入力します．
この分類モデルはカルテのあるtokenがfeatureに関して重要な記述であるか，つまりannotationに含まれるかどうかを判定（0 or 1）します．
カルテのすべてのtokenに対して判定を行い，連続して重要であると判定されたtokenをまとめて1つの予測annotationを作ります．

つまりDataset作成では入力としてfeature_textとカルテのテキスト（それぞれをtokenizeしたもの）と，ラベルとしてカルテの各tokenがannotationに含まれているかどうかを表す0 or 1のリストを作成します．

## 前処理

3つのcsvファイルを読み込んでマージし，次の前処理を行います．

1. `ast.literal_eval`で`trian.csv`のannotationとlocationを文字列からリストに変換
1. feature_textに含まれている`"-OR-"`を`"; "`に，`"-"`を`" "`に置き換える
1. feature_textとpn_history（カルテ文）に含まれれる文字をすべて小文字に変換．

今回利用するモデルはuncasedのBERTなので，大文字と小文字を区別しません．

In [ ]:
BASE_URL = "../input/nbme-score-clinical-patient-notes"


def process_feature_text(text):
    return text.replace("-OR-", "; ").replace("-", " ")


def prepare_datasets():
    features = pd.read_csv(f"{BASE_URL}/features.csv")
    notes = pd.read_csv(f"{BASE_URL}/patient_notes.csv")
    train = pd.read_csv(f"{BASE_URL}/train.csv")

    merged = train.merge(notes, how="left")
    merged = merged.merge(features, how="left")

    merged["annotation_list"] = [literal_eval(x) for x in merged["annotation"]]
    merged["location_list"] = [literal_eval(x) for x in merged["location"]]
    
    merged["feature_text"] = [process_feature_text(x) for x in merged["feature_text"]]
    
    merged["feature_text"] = merged["feature_text"].apply(lambda x: x.lower())
    merged["pn_history"] = merged["pn_history"].apply(lambda x: x.lower())

    return merged

train_df = prepare_datasets()
train_df.head()

## Dataset作成

`train_df`からPytorch Datasetを作成します．最初に説明したように入力としてfeature_textとカルテのテキスト（それぞれをtokenizeしたもの）と，ラベルとしてカルテのテキストのtokenがannotationに含まれているかどうかを表す0 or 1のリストを作成します．

複雑な処理なので，まずは次のような簡単な例を使って処理の流れを説明します．

In [ ]:
example = train_df.loc[14242]
example

## token分割

まずtransformersのTokenizerでpn_historyとfeature_textをtokenに分割します．

本番submit時にはネットワークが使えないらしく，HuggingFace Hubからモデルをダウンロードできないため，Kaggleにある[Huggingface BERT](https://www.kaggle.com/xhlulu/huggingface-bert)をNotebookに追加して使用します．（右のタブの"+Add data"から追加します）


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("../input/huggingface-bert/bert-base-uncased")

# submitしないのであれば，Hubから直接ダウンロードできる
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizerには文のペアを渡すことができます．ここではfeature_textとpn_historyをこの順番でtokenizerに渡してtokenに変換します．

[Transformers Docs](https://huggingface.co/docs/transformers/preprocessing)

私が気になったのは，`max_length=416`としている点です．ここで利用している[bert_base_uncasedのモデルカード](https://huggingface.co/bert-base-uncased#:~:text=The%20only%20constrain%20is%20that%20the%20result%20with%20the%20two%20%22sentences%22%20has%20a%20combined%20length%20of%20less%20than%20512%20tokens.)には「２つの文を合わせた長さは最大512tokenに制限される」と書かれていますが，ここではなぜかそれよりも短い416という値を使っています．

In [ ]:
out = tokenizer(example["feature_text"], example["pn_history"], max_length=416, truncation="only_second", padding="max_length", return_offsets_mapping=True)
out.keys()

input_idsはfeature_textとpn_historyをtokenに分割したものです．tokenそのものではなくtokenのidが入っています．

`tokenizer.convert_ids_to_tokens`でtoken idをtokenに変換できます．token idをtokenに戻してみるともとの文が確認できます．
feature_textとpn_historyは結合され`[SEP]`で区切られています．

In [ ]:
print(len(out["input_ids"]))
print(out["input_ids"])
print(tokenizer.convert_ids_to_tokens(out["input_ids"]))

tokenizerで`tokenizer(..., return_offsets_mapping=True)`としていたので，tokenに変換した際にoffset_mappingも帰ってきます．

offset_mappingを見ると，各tokenがもとの文のどの部分に対応するかがわかります．
例えば"lives"は11番目のtokenで，offset_mappingは(13, 18)となっています．もとの文（pn_history）で13文字目から17文字目までを見ると，確かに"lives"となっています．

In [ ]:
print(out["offset_mapping"][:15]) #長いので最初の15個を表示
print(example["pn_history"][13:18])

`out.sequence_ids()`を見ることで，各tokenが入力した２つの文（feature_text，pn_history）のどちらから作られたかを知ることができます．

416個の各tokenについて，`0`はfeature_text，`1`はpn_historyに属しているtokenであることを表しています．`None`となっているのは`[SEP]`や`[PAD]`などの特殊なtokenです．

`out["sequence_ids"]`に入れておきます．

In [ ]:
print(len(out.sequence_ids()))
print(out.sequence_ids())

out["sequence_ids"] = out.sequence_ids()

## ラベル作成

次に，annotaionの位置`example["location_list"]`をもとにしてラベルを作成します．

前処理で文字列からリストへ変換しましたが，まだ中身は文字列なのでさらに変換を行います．`loc_list_to_ints`という関数で変換を行います．
また変換したものを`out["location_int"]`に入れておきます．

In [ ]:
def loc_list_to_ints(loc_list):
    to_return = []
    for loc_str in loc_list:
        loc_strs = loc_str.split(";")
        for loc in loc_strs:
            start, end = loc.split()
            to_return.append((int(start), int(end)))
    return to_return


print(f"{example['location_list']} -> {loc_list_to_ints(example['location_list'])}")

print(f"{['682 688;695 697']} -> {loc_list_to_ints(['682 688;695 697'])}")


out["location_int"] = loc_list_to_ints(example["location_list"])

`out["sequence_ids"]`と`out["offset_mapping"]`，そして`out["location_int"]`を使って，各tokenにがfeatureに関する重要な記述であるかどうか（annotationに含むまれるか）を表すラベルを作成します．

In [ ]:
labels = [0.0] * len(out["input_ids"]) # tokenの数と同じ長さのラベルのリストを用意

# 各tokenのseq_id（どちらの文に属しているか）とoffsets（もとの文のどの範囲にあるか）
for idx, (seq_id, offsets) in enumerate(zip(out["sequence_ids"], out["offset_mapping"])):
    # tokenが特殊なtoken（[PAD]や[SEQ]）である場合，あるいは１つ目の文（feature_text）に属している場合はラベルを付けたくないので-1としておく．
    # （ラベルが-1のtokenは，後でlossを計算する際に無視されます）
    if not seq_id or seq_id == 0:
        labels[idx] = -1
        continue

    # tokenがannotaionの範囲に含まれている場合，そのtokenのラベルを1.0にする．
    token_start, token_end = offsets
    for feature_start, feature_end in out["location_int"]:
        if token_start >= feature_start and token_end <= feature_end:
            labels[idx] = 1.0
            break

out["labels"] = labels

print(out["labels"])

これでデータセットを作成できます．改めてコードをまとめます．

In [ ]:
hyperparameters = {
    "max_length": 416,
    "padding": "max_length",
    "return_offsets_mapping": True,
    "truncation": "only_second",
    "model_name": "../input/huggingface-bert/bert-base-uncased",
    "dropout": 0.2,
    "lr": 1e-5,
    "test_size": 0.2,
    "seed": 1268,
    "batch_size": 8
}

def loc_list_to_ints(loc_list):
    to_return = []
    for loc_str in loc_list:
        loc_strs = loc_str.split(";")
        for loc in loc_strs:
            start, end = loc.split()
            to_return.append((int(start), int(end)))
    return to_return


def tokenize_and_add_labels(tokenizer, data, config):
    out = tokenizer(
        data["feature_text"],
        data["pn_history"],
        truncation=config['truncation'],
        max_length=config['max_length'],
        padding=config['padding'],
        return_offsets_mapping=config['return_offsets_mapping']
    )
    labels = [0.0] * len(out["input_ids"])
    out["location_int"] = loc_list_to_ints(data["location_list"])
    out["sequence_ids"] = out.sequence_ids()

    for idx, (seq_id, offsets) in enumerate(zip(out["sequence_ids"], out["offset_mapping"])):
        if not seq_id or seq_id == 0:
            labels[idx] = -1
            continue

        token_start, token_end = offsets
        for feature_start, feature_end in out["location_int"]:
            if token_start >= feature_start and token_end <= feature_end:
                labels[idx] = 1.0
                break

    out["labels"] = labels

    return out

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, config):
        self.data = data
        self.tokenizer = tokenizer
        self.config = config

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data.iloc[idx]
        tokens = tokenize_and_add_labels(self.tokenizer, data, self.config)

        input_ids = np.array(tokens["input_ids"])
        attention_mask = np.array(tokens["attention_mask"])
        token_type_ids = np.array(tokens["token_type_ids"])

        labels = np.array(tokens["labels"])
        offset_mapping = np.array(tokens['offset_mapping'])
        sequence_ids = np.array(tokens['sequence_ids']).astype("float16")
        
        return input_ids, attention_mask, token_type_ids, labels, offset_mapping, sequence_ids

dataloaderを作成します．

In [ ]:
train_df = prepare_datasets()

X_train, X_test = train_test_split(train_df, test_size=hyperparameters['test_size'],
                                   random_state=hyperparameters['seed'])

print("Train size", len(X_train))
print("Test Size", len(X_test))

tokenizer = AutoTokenizer.from_pretrained(hyperparameters['model_name'])

training_data = CustomDataset(X_train, tokenizer, hyperparameters)
train_dataloader = DataLoader(training_data, batch_size=hyperparameters['batch_size'], shuffle=True)

test_data = CustomDataset(X_test, tokenizer, hyperparameters)
test_dataloader = DataLoader(test_data, batch_size=hyperparameters['batch_size'], shuffle=False)

試しにbatchを１つ取り出してみます．

In [ ]:
for batch in train_dataloader:
    input_ids, attention_mask, token_type_ids, labels, offset_mapping, sequence_ids = batch
    print(input_ids.shape)
    break

# Model

次にモデルを作成します．BERTの出力にFC層をつなげたモデルになっています．

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.bert = AutoModel.from_pretrained(config['model_name'])  # BERT model
        self.dropout = nn.Dropout(p=config['dropout'])
        self.config = config
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = self.fc1(outputs[0])
        logits = self.fc2(self.dropout(logits))
        logits = self.fc3(self.dropout(logits)).squeeze(-1)
        return logits

モデルに入力されたテンソルの形がどのように変わっていくかを見てみます．

BERTの出力の形は`(batch_size, n_tokens, hidden_dim) = (8, 416, 768)`で，FC層を通すことで最終的に`(8, 416)`の形になります．各batchの各tokenに対して１つの値（logit）が出力されています．これをsigmoidに通すことで確率を計算します．

モデル作成時に"Some weights of the ..."という警告が出ていますが正常です．[参考](https://huggingface.co/course/chapter3/3?fw=pt#:~:text=You%20will%20notice,to%20do%20now.)

In [ ]:
model = CustomModel(hyperparameters)

for batch in train_dataloader:
    input_ids, attention_mask, token_type_ids, labels, offset_mapping, sequence_ids = batch
    bert_output = model.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    print(bert_output.last_hidden_state.size())
    
    logits = model.fc1(bert_output[0]) # bert_output[0]とbert_output.last_hidden_stateは同じ
    logits = model.fc2(model.dropout(logits))
    logits = model.fc3(model.dropout(logits))
    print(logits.size())
    
    logits = logits.squeeze(-1)
    print(logits.size())
    break

# Training

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss(reduction = "none")

def train_model(model, dataloader, optimizer, criterion):
        model.train()
        train_loss = []

        for batch in tqdm(dataloader):
            optimizer.zero_grad()
            input_ids = batch[0].to(DEVICE)
            attention_mask = batch[1].to(DEVICE)
            token_type_ids = batch[2].to(DEVICE)
            labels = batch[3].to(DEVICE)

            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits, labels)
            
            loss = torch.masked_select(loss, labels > -1.0).mean()
            train_loss.append(loss.item() * input_ids.size(0))
            loss.backward()
            # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
            # it's also improve f1 accuracy slightly
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        return sum(train_loss)/len(train_loss)

先ほどと同様に`train_dataloader`からbatchを１つ取り出して流れを確認します．

-1.0のラベルを付けたtoken（feature_textに属している or 特殊なtoken）に対するモデルのlossは訓練に使いたくないため，`torch.masked_select`でフィルタリングしています．

また`BCEWithLogitsLoss`で`reduction="none"`とすることで，各tokenに対するlossを計算できます．

In [ ]:
for batch in train_dataloader:
    input_ids, attention_mask, token_type_ids, labels, offset_mapping, sequence_ids = batch

    logits = model(input_ids, attention_mask, token_type_ids)
    print(logits.size())
    
    # criterion = BCEWithLogitsLoss(reduction = "none") とすることで，各batchの各tokenのlossがわかる．
    # デフォルトではreduction = "mean"
    loss = criterion(logits, labels)
    print(loss.size())

    # ラベル作成時に-1.0とした部分のlossは無視する．
    loss = torch.masked_select(loss, labels > -1.0)
    print(loss.size()) # ラベルが 0 or 1 のlossのみがのこる．
    
    print(loss.mean())
    
    break

In [ ]:
# ラベルが 0 or 1 のlossの数
np.sum(labels.numpy()>-1.0)

🤗 Transformersには訓練に必要な処理をまとめたTrainerクラスが存在しますが，今回は上で説明したような処理を組み込む必要があったため使用していないようです．

次に評価用の関数です．ほとんど訓練用の関数と同じですが，token単位の予測結果を文字単位の範囲に変換する処理とmetricの計算が追加されています．

[Evaluation方法](https://www.kaggle.com/c/nbme-score-clinical-patient-notes/overview/evaluation)

In [ ]:
def eval_model(model, dataloader, criterion):
        model.eval()
        valid_loss = []
        preds = []
        offsets = []
        seq_ids = []
        valid_labels = []

        for batch in tqdm(dataloader):
            input_ids = batch[0].to(DEVICE)
            attention_mask = batch[1].to(DEVICE)
            token_type_ids = batch[2].to(DEVICE)
            labels = batch[3].to(DEVICE)
            offset_mapping = batch[4]
            sequence_ids = batch[5]

            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits, labels)
            loss = torch.masked_select(loss, labels > -1.0).mean()
            valid_loss.append(loss.item() * input_ids.size(0))

            preds.append(logits.detach().cpu().numpy())
            offsets.append(offset_mapping.numpy())
            seq_ids.append(sequence_ids.numpy())
            valid_labels.append(labels.detach().cpu().numpy())

        preds = np.concatenate(preds, axis=0)
        offsets = np.concatenate(offsets, axis=0)
        seq_ids = np.concatenate(seq_ids, axis=0)
        valid_labels = np.concatenate(valid_labels, axis=0)
        location_preds = get_location_predictions(preds, offsets, seq_ids, test=False)
        score = calculate_char_cv(location_preds, offsets, seq_ids, valid_labels)

        return sum(valid_loss)/len(valid_loss), score

    
# token単位の予測結果を文字単位の範囲に変換する．
def get_location_predictions(preds, offset_mapping, sequence_ids, test=False):
    all_predictions = []
    for pred, offsets, seq_ids in zip(preds, offset_mapping, sequence_ids):
        pred = 1 / (1 + np.exp(-pred)) # logitsからprobabilityを計算
        
        start_idx = None
        end_idx = None
        current_preds = []
        for pred, offset, seq_id in zip(pred, offsets, seq_ids):
            if seq_id is None or seq_id == 0:
                continue

            # probability > 0.5 が連続している部分をtoken単位で探し，
            # そのstart位置とstop位置の（文字単位での）idxを記録する．
            if pred > 0.5:
                if start_idx is None:
                    start_idx = offset[0]
                end_idx = offset[1]
            elif start_idx is not None:
                if test:
                    current_preds.append(f"{start_idx} {end_idx}")
                else:
                    current_preds.append((start_idx, end_idx))
                start_idx = None
        if test:
            all_predictions.append("; ".join(current_preds))
        else:
            all_predictions.append(current_preds)
            
    return all_predictions


# 文字単位で評価値を計算する．
def calculate_char_cv(predictions, offset_mapping, sequence_ids, labels):
    all_labels = []
    all_preds = []
    for preds, offsets, seq_ids, labels in zip(predictions, offset_mapping, sequence_ids, labels):

        num_chars = max(list(chain(*offsets)))
        char_labels = np.zeros(num_chars)

        # ラベル
        for o, s_id, label in zip(offsets, seq_ids, labels):
            if s_id is None or s_id == 0:
                continue
            if int(label) == 1:
                char_labels[o[0]:o[1]] = 1

        char_preds = np.zeros(num_chars)

        # 予測結果
        for start_idx, end_idx in preds:
            char_preds[start_idx:end_idx] = 1

        all_labels.extend(char_labels)
        all_preds.extend(char_preds)

    results = precision_recall_fscore_support(all_labels, all_preds, average="binary", labels=np.unique(all_preds))
    accuracy = accuracy_score(all_labels, all_preds)
    

    return {
        "Accuracy": accuracy,
        "precision": results[0],
        "recall": results[1],
        "f1": results[2]
    }

実際に訓練を行うコードです．（実行に時間がかかるのでコメントアウトしています）

In [ ]:
# train_df = prepare_datasets()
# X_train, X_test = train_test_split(train_df, test_size=hyperparameters['test_size'], random_state=hyperparameters['seed'])

# tokenizer = AutoTokenizer.from_pretrained(hyperparameters['model_name'])
# training_data = CustomDataset(X_train, tokenizer, hyperparameters)
# train_dataloader = DataLoader(training_data, batch_size=hyperparameters['batch_size'], shuffle=True)
# test_data = CustomDataset(X_test, tokenizer, hyperparameters)
# test_dataloader = DataLoader(test_data, batch_size=hyperparameters['batch_size'], shuffle=False)

# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# model = CustomModel(hyperparameters).to(DEVICE)
# criterion = torch.nn.BCEWithLogitsLoss(reduction = "none")
# optimizer = optim.AdamW(model.parameters(), lr=hyperparameters['lr'])

# train_loss_data, valid_loss_data = [], []
# score_data_list = []
# valid_loss_min = np.Inf
# since = time.time()
# epochs = 3

# best_loss = np.inf

# for i in range(epochs):
#     print("Epoch: {}/{}".format(i + 1, epochs))
#     # first train model
#     train_loss = train_model(model, train_dataloader, optimizer, criterion)
#     train_loss_data.append(train_loss)
#     print(f"Train loss: {train_loss}")
#     # evaluate model
#     valid_loss, score = eval_model(model, test_dataloader, criterion)
#     valid_loss_data.append(valid_loss)
#     score_data_list.append(score)
#     print(f"Valid loss: {valid_loss}")
#     print(f"Valid score: {score}")
    
#     if valid_loss < best_loss:
#         best_loss = valid_loss
#         torch.save(model.state_dict(), "nbme_bert_v2.pth")

    
# time_elapsed = time.time() - since
# print('Training completed in {:.0f}m {:.0f}s'.format(
#     time_elapsed // 60, time_elapsed % 60))